In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.fft as fft
import plotly.graph_objects as go
import plotly.express as px
import webbrowser
from scipy.optimize import curve_fit
import os
import re
import sympy as sp

In [2]:
AGG_IN_MIN = 70
AGG_IN_MAX = 160

KP_A = 0.04
KP_B = 0.3

KD_A = 0.001
KD_B = 1.0

In [3]:
def aggIn(KP, KD):
    """This function computes an approximation of the aggressiveness index based on the KP and KD values of the PID controller,
    while considering the KI value to be 0.01. The equation has been obtained by fitting a polynomial of degree 3 to the data obtained
    from the simulation of the PID controller.

    Args:
        KP : float
            Proporcional term of the PID controller.
        KD : float
            Derivative term of the PID controller.

    Returns:
        out : float
            Approximation of the aggressiveness index of the PID controller if it would be used to follow FTP-75 driving cycle.
    """
    
    args = [ 34.20949562, 1317.99024017, -156.27590486, -544.54377573, -5732.4692288 , 282.56780882, 9396.56391293, -148.9753875, -51.41313683, 1244.93126392]
    
    return args[0] + args[1]*KP + args[2]*KD + args[3]*KP*KD + args[4]*KP**2 + args[5]*KD**2 + args[6]*KP**3 + args[7]*KD**3 + args[8]*KP*KD**2 + args[9]*KP**2*KD

In [4]:
def esd(yt, fs):
    """Computes the Energy Spectral Density of a signal yt, keeping only positive frequencies (note that this means that the total energy is halved). 
    
    Parameters
    ----------
        yt : array_like 
            Signal to be analyzed
        fs : float 
            Sampling frequency of the signal

    Returns
    -------
        out : array_like, array_like
            Frequency axis and Energy Spectral Density of the signal
    """
    N = len(yt)
    f = fft.rfftfreq(N, 1/fs)
    Sxx = np.square(1 / fs) *  np.square(np.abs(fft.rfft(yt)))
    
    return f, Sxx

In [5]:
def possible_PIDs(target_aggIn, k = 10):
    """Provides a list of at most k possible PID controllers that satisfy the target aggressiveness index.
    To do so it generates a random KP from a normal distribution with mean depending on the target aggressiveness index 
    and then finds the possible KD values that satisfy the target aggressiveness index with the current KP.
    The solutions are filtered to keep only the real ones, that are in the range and that are not too close to the ones already in the list.
    The number of elements in the list can be smaller than k to speed up the process when the loop keeps finding the too similar solutions.

    Parameters
    ----------
        target_aggIn : float 
            The aggressiveness index to satisfy (must be between AGG_IN_MIN=70 and AGG_IN_MAX=160).
        k : int, optional
            The maximum number of solutions to return. Defaults to 10.

    Returns
    -------
        out: list
            List of at most k possible PID controllers that satisfy the target aggressiveness index.

    Raises
    -------
        ValueError
            When aggressiveness index is out of range (must be between 0 and 1).
    """


    if target_aggIn < AGG_IN_MIN or target_aggIn > AGG_IN_MAX:
        raise ValueError("Aggressiveness index out of range")
    
    MAX_ITER = 300
    PRECISION_on_KP = 0.003
    
    res_list = []
    i = 0
    
    
    while (len(res_list) < k or (i < MAX_ITER and len(res_list) <= 1)):
        # generate KP from a random normal distribution with mean depending on normalized aggressiveness index   
        KP = 0
        while(KP < KP_A or KP > KP_B):
            KP = np.random.normal((KP_B - KP_A) * (1 + 2 * (target_aggIn - AGG_IN_MIN) / (AGG_IN_MAX - AGG_IN_MIN)) / 4, (KP_B - KP_A) / 3)
            
        # find possible KD values that sarisfy the target aggressiveness index with the current KP
        KD = sp.Symbol('KD')
        eq = sp.Eq(aggIn(KP, KD), target_aggIn)
        sol = sp.solve(eq, KD)
        
        # filter the solutions to keep only the real ones and the ones in the range
        sol = [s for s in sol if s.is_real and s >= KD_A and s <= KD_B]
        
        # append the solutions to the result list only if they are not too close to the ones already in the list
        for s in sol:
            if len(res_list) < 1 or len([d for d in res_list if abs(d['KP'] - KP) < PRECISION_on_KP]) == 0:
                res_list.append({'KP' : float(KP), 'KI' : 0.01, 'KD' : float(s)}) 
            else:
                k -= 1

        i += 1
        
    return res_list

In [87]:
def bestPID(target_aggIn = 107):
    """Given a target aggressiveness index, it returns the best PID controller that satisfies it.
    To do so it computers a list of possible PIDs calling the `possible_PIDs()` function and then it finds the one that has a KP parameter
    closest to the center of the range shifted according to relative aggIn of possible PIDs, by calculating the euclidean normalized distance from the center.

    Args:
        target_aggIn (float, optional): the aggressiveness index to satisfy. Defaults to 107.

    Raises:
        ValueError: aggressiveness index out of range (must be between 0 and 1)

    Returns:
        dict: dictionary containing the KP, KI, KD parameters of the best PID controller that satisfies the target aggressiveness index
    """
    
    
    if target_aggIn < AGG_IN_MIN or target_aggIn > AGG_IN_MAX:
        raise ValueError("AggressivenesKD index out of range")
    
    
    KP_center = (KP_B - KP_A) / 2
    shifted_KP_center = KP_A + KP_center + (KP_B - KP_A) * ((target_aggIn - AGG_IN_MIN) / (AGG_IN_MAX - AGG_IN_MIN) - 0.5) * 0.8
    
    min_dist = float('inf')
    best_PID = {}
    
    possible_PIDs_list = possible_PIDs(target_aggIn, 10)
    
    for PID in possible_PIDs_list:
        dist = np.abs(shifted_KP_center - PID['KP'])
        if dist < min_dist:
            min_dist = dist
            best_PID.clear()
            best_PID.update(PID)    
    
    return best_PID

In [104]:
res = possible_PIDs(155)
best = bestPID(155)

In [105]:
best

{'KP': 0.2636911942160516, 'KI': 0.01, 'KD': 0.0020881428167307606}